*[Importing](https://www.kaggle.com/competitions/fake-news/data) the data and dependency

In [1]:
import nltk
stopwords=nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nandan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import numpy as np
import pandas as pd
import re #regex
# natural language tool kit
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
# scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Data preprocessing


In [3]:
#loading the dataset into data frame
news_df=pd.read_csv("./train.csv")
news_df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [4]:
news_df.shape

(20800, 5)

In [5]:
news_df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
news_df=news_df.dropna()

In [7]:
news_df.shape

(18285, 5)

as out of 20k we are getting back 18k rows
it should be good enough

if many rows were missing them we could use methods like imputation 

#### we shall consider the title and author of detecting the news is fake or not 
we could have used the text i.e is the body of the news but the body could be really long that would dramtically increase the training time 
also its the title of the article which attracts the viewers to click on that fake news after that they read the content of it

In [8]:
# merging the author name and title and text i.e body of the news article

news_df['content']=news_df['title']+" "+news_df['author']+news_df['text']
news_df.head(1)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...


In [9]:
# seperating the independant variables and op variable
X=news_df['content']
X.head(1)

0    House Dem Aide: We Didn’t Even See Comey’s Let...
Name: content, dtype: object

In [10]:
y=news_df[['label']]
y.head(1)

,label
0,1


Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [11]:
port_stem = PorterStemmer()


In [12]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # we just want words without special characters and digits
    stemmed_content = stemmed_content.lower() # make it  all lower
    stemmed_content = stemmed_content.split() # covert string to arr for the below line
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # remove stop words like is and at these are insignificant words that are used for sentense formatation
    # they are not good keywords
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [ ]:
X=X.apply(stemming)

In [62]:
X.shape

(18285, 113933)

In [15]:
X.head(1) # can notice all stop words are gone
# no special characters no nuumbers
# and also we have reduced every word to its base word

0    hous dem aid even see comey letter jason chaff...
Name: content, dtype: object

#### TfidVectorizer is very similar to that of CountVectorizer 
(bag of words approach to convert the textual data to vectors)
Bag-of-Words:
##### The bag-of-words model converts text into fixed-length vectors by counting how many times each word appears.


##### Term Frequency Inverse Document Frequency (TFIDF) :
TFIDF works by proportionally increasing the number of times a word appears in the document but is counterbalanced by the number of documents in which it is present. Hence, words like ‘this’, ’are’ etc., that are commonly present in all the documents are not given a very high rank. However, a word that is present too many times in a few of the documents will be given a higher rank as it might be indicative of the context of the document.

In [16]:
# converting the textual data to numberical data

vectorizer = TfidfVectorizer()
#Docstring:     
#Convert a collection of raw documents to a matrix of TF-IDF features.
#Equivalent to :class:`CountVectorizer` followed by
#:class:`TfidfTransformer`.
vectorizer.fit(X)
X=vectorizer.transform(X)
print(X)

  (0, 112261)	0.049266288431976044
  (0, 112203)	0.018822028602683175
  (0, 111235)	0.044332109417032826
  (0, 111231)	0.0952049285944191
  (0, 111188)	0.037590717156198834
  (0, 111151)	0.011177780002938412
  (0, 110412)	0.017156396746647547
  (0, 109587)	0.017030258214578126
  (0, 109483)	0.012515568355693731
  (0, 109390)	0.029807510521693708
  (0, 109314)	0.012677958954410361
  (0, 109094)	0.01164029909301929
  (0, 108202)	0.025650088468419645
  (0, 108168)	0.03154160796361522
  (0, 107174)	0.0216800693516796
  (0, 105849)	0.06552976152279262
  (0, 105187)	0.033407039293133704
  (0, 104931)	0.016231190325023088
  (0, 104181)	0.03819403882577124
  (0, 103565)	0.010943506714122472
  (0, 103556)	0.042297456119198974
  (0, 103503)	0.13826067644470952
  (0, 103346)	0.07074511938458296
  (0, 102115)	0.03965402110564886
  (0, 101563)	0.027042557452073717
  :	:
  (18284, 7564)	0.010375309794254967
  (18284, 7250)	0.027544993804815406
  (18284, 6942)	0.03863683979084329
  (18284, 6904)	0.02

In [17]:
x_train,x_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.2)

# staritfy is used to maintain the balance or ration of y being 1 and 0

#Some classification problems do not have a balanced number of examples for each class label. 
#As such, it is desirable to split the dataset into train and test sets in a way that preserves 
#the same proportions of examples in each class as observed in the original dataset. 
#This is called a stratified train-test split


In [18]:
logisticReg= LogisticRegression()
# signmoid func -> S curve 
#y=1/1+e^-z 
# z->st line -> z=wX+b
# X->content-> input feature
# y->predict prob
# w->weigts -> imp of feature
# b->bias ->intercept
# rule of thumb 

In [19]:
logisticReg.fit(x_train,y_train)

/Users/nandan/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [20]:
logisticReg.score(x_test,y_test)

0.9581624282198523

In [21]:
logisticReg.score(x_train,y_train)

0.9780557834290402

In [22]:
print(x_test[1])

  (0, 112552)	0.0025446087965112657
  (0, 111188)	0.002897775546205445
  (0, 111069)	0.009709433163818147
  (0, 110987)	0.0039602820168925055
  (0, 110545)	0.012893685371874849
  (0, 110031)	0.004621428900345242
  (0, 109874)	0.003526757032611399
  (0, 109483)	0.008683148288861587
  (0, 108258)	0.007366295085852576
  (0, 107717)	0.10030367892392501
  (0, 106523)	0.02066743126938718
  (0, 106518)	0.05185025506624489
  (0, 106331)	0.00623858505778796
  (0, 106185)	0.004681893823375919
  (0, 105871)	0.006702812334919964
  (0, 105760)	0.002516659194516403
  (0, 105660)	0.008418963967750229
  (0, 104254)	0.009426570841073166
  (0, 103639)	0.005424474139949627
  (0, 103626)	0.010143140756594329
  (0, 103359)	0.023802702939388243
  (0, 102597)	0.009948756302141808
  (0, 102487)	0.003030800215979766
  (0, 102380)	0.01011271272409316
  (0, 101986)	0.008972840574635198
  :	:
  (0, 6108)	0.03260789374884701
  (0, 5794)	0.004043894645879154
  (0, 5032)	0.052257937214600515
  (0, 5004)	0.0139787852

the acuracy score is pretty much the same
this means our model doesn't try to over fit the training data

 ### making a predictive system 

In [23]:
X_new = x_test[1]
prediction = logisticReg.predict(X_new)
print(prediction)

[1]


In [68]:
import pickle
fp = open("fake-news-model.pickle","wb") # file pointer
fp2 = open("vectorizer.pickle","wb") # file pointer
pickle.dump(logisticReg,fp)
pickle.dump(vectorizer,fp2)
fp.close()

In [25]:
def convert_text_into_numbers(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # we just want words without special characters and digits
    stemmed_content = stemmed_content.lower() # make it  all lower
    stemmed_content = stemmed_content.split() # covert string to arr for the below line
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # remove stop words like is and at these are insignificant words that are used for sentense formatation
    # they are not good keywords
    stemmed_content = ' '.join(stemmed_content)
    vectorizer = TfidfVectorizer()
    #Docstring:     
    #Convert a collection of raw documents to a matrix of TF-IDF features.
    #Equivalent to :class:`CountVectorizer` followed by
    #:class:`TfidfTransformer`.
    vectorizer.fit(stemmed_content)
    X=vectorizer.transform(stemmed_content)
    predict = logisticReg.predict(X)
    return predict 
    

In [66]:
X.shape

(18285, 113933)

<18285x113933 sparse matrix of type '<class 'numpy.float64'>'
	with 4776977 stored elements in Compressed Sparse Row format>

In [65]:
arr=[]
for i in X:
    print(len(i.data))
    print(i.)

    
    

260
275
440
175
66
85
297
486
416
184
198
104
1238
330
384
177
375
108
127
69
250
251
192
175
241
1496
165
393
491
345
553
864
887
136
264
194
247
208
172
331
199
153
143
307
137
270
108
423
356
597
383
23
404
153
296
181
456
126
128
76
352
1565
142
493
425
260
106
365
83
403
29
6
117
517
104
20
437
123
304
321
216
394
376
80
231
92
421
356
23
349
280
192
136
194
185
143
263
422
99
72
387
404
195
266
341
387
480
270
357
370
38
648
486
224
190
51
60
515
357
120
287
201
146
210
194
133
360
140
50
252
61
160
178
421
473
325
418
410
125
1054
452
315
25
26
110
248
341
346
88
12
71
103
10
239
363
435
617
298
576
278
435
262
281
242
119
342
40
1587
142
167
609
1126
632
458
178
462
138
245
117
386
325
160
523
187
56
56
18
385
242
357
489
165
353
494
234
53
464
393
186
360
104
395
274
116
433
151
216
67
123
114
361
50
277
242
271
411
96
170
38
201
93
363
384
457
140
399
475
306
447
146
47
420
171
517
176
361
81
482
47
450
99
207
263
139
30
86
307
440
376
31
277
25
257
56
287
10
302
291
781
212


102
501
335
77
676
428
572
269
215
229
85
297
116
127
118
238
286
170
254
511
554
285
55
136
23
68
277
593
184
271
55
200
135
284
54
290
29
88
499
334
273
286
81
239
249
159
461
86
301
248
108
288
3
372
78
169
551
424
131
172
181
264
338
487
199
36
382
376
288
369
277
403
355
385
126
311
306
537
361
130
97
207
8
280
361
138
317
1308
125
228
79
79
451
499
248
173
421
155
149
487
203
181
125
65
409
486
424
142
171
148
147
302
307
117
191
366
462
451
25
363
262
99
128
178
84
1606
117
321
291
518
330
121
214
245
547
282
452
118
302
116
63
441
303
376
306
301
64
474
81
183
102
114
177
126
97
150
52
36
131
421
268
417
392
184
182
75
60
146
276
142
417
93
1572
83
157
183
248
192
271
73
282
183
268
317
91
499
256
268
183
437
40
73
500
82
163
125
194
45
66
110
328
346
170
82
555
201
472
87
238
45
292
377
22
149
279
294
38
372
140
154
551
97
36
220
115
191
375
100
366
162
361
239
403
700
126
87
12
598
453
379
88
96
76
315
380
57
778
354
277
141
234
172
72
29
225
374
134
288
369
11
89
88
110
146


412
23
117
22
115
187
313
361
106
239
72
443
15
337
259
249
178
440
333
165
249
217
376
204
155
360
356
184
299
207
29
14
63
252
145
487
236
74
324
305
263
67
528
261
271
56
290
179
377
344
372
244
230
205
190
337
198
310
362
156
424
215
260
129
250
413
276
163
41
46
429
134
95
158
434
14
282
875
307
103
113
103
173
179
30
292
552
240
372
293
349
793
173
523
600
200
138
185
405
90
25
375
295
152
304
92
61
244
129
541
46
321
417
317
71
289
43
190
138
373
93
102
354
34
248
146
386
458
593
181
186
397
165
363
499
279
392
473
1579
456
359
87
115
277
285
465
157
92
196
5
439
52
122
397
22
37
283
71
358
283
209
344
195
116
107
69
179
41
105
390
109
317
123
174
109
99
1233
94
585
119
31
121
177
257
224
145
119
453
368
62
850
296
185
53
291
153
207
109
330
343
108
394
394
215
759
331
149
275
275
571
402
106
881
23
370
492
183
3
456
411
328
186
440
105
159
180
64
288
178
439
51
363
482
61
366
445
473
216
400
384
148
133
494
42
173
562
136
366
73
404
234
82
340
89
454
122
351
74
197
167
70
116
4

113
140
110
344
382
156
268
72
228
144
281
324
230
405
78
1106
222
180
131
134
161
143
84
138
340
100
101
124
1324
216
135
162
8
476
320
363
241
175
400
451
27
135
113
382
178
128
477
181
132
6
366
428
84
100
87
64
132
294
762
247
405
213
137
517
261
179
287
153
1356
98
247
181
439
183
453
470
296
157
150
304
127
34
537
393
56
259
349
413
336
418
1014
279
432
398
120
256
390
127
226
99
62
441
123
145
57
189
592
291
109
72
74
552
130
127
61
225
91
165
23
247
237
100
132
35
296
200
254
262
305
456
312
447
277
159
271
250
389
474
258
403
75
387
178
151
33
307
301
224
104
212
561
408
418
382
81
68
343
244
311
356
348
280
182
436
78
246
346
212
415
331
159
116
178
168
108
283
86
108
377
281
347
214
225
171
546
322
74
290
413
271
218
180
73
191
258
176
121
140
499
104
132
47
69
41
165
125
195
129
145
414
613
155
167
309
144
104
12
280
164
419
220
268
130
216
380
81
149
184
106
318
457
528
360
103
490
303
93
353
183
341
144
372
108
134
71
415
119
241
354
483
344
44
59
198
10
360
214
108
99
21

431
261
70
304
32
38
210
271
385
904
93
105
186
73
247
76
269
188
178
406
167
272
675
354
169
532
25
535
405
72
478
424
535
166
474
126
70
468
95
113
580
78
369
256
263
359
190
166
183
163
211
462
385
617
93
210
240
309
98
477
194
368
382
90
93
70
374
45
341
90
442
98
316
214
173
93
403
343
116
391
209
341
690
210
143
207
260
353
389
116
381
187
208
294
241
393
308
565
255
213
852
438
38
215
247
188
298
88
434
528
160
164
119
222
193
332
38
489
345
87
200
165
307
310
216
382
49
67
53
131
399
96
366
297
69
594
413
142
45
149
369
141
107
201
228
12
218
223
84
144
449
133
151
54
116
211
124
484
70
351
258
244
334
225
142
22
280
359
73
990
67
533
362
226
235
388
493
201
250
34
254
216
122
29
766
210
85
199
333
327
306
281
97
384
216
473
54
253
133
287
135
140
468
331
140
122
300
261
177
164
405
71
466
337
231
4
191
223
257
109
290
286
256
434
387
359
107
429
74
155
147
752
405
476
75
491
397
604
225
202
78
403
339
107
9
599
374
661
316
250
154
452
393
14
424
190
142
426
230
161
208
163
270

93
450
849
198
383
353
246
144
213
356
392
295
286
353
386
92
287
220
199
64
333
105
338
382
217
224
289
252
69
341
430
39
479
23
100
105
509
39
373
111
209
124
156
340
485
383
281
265
79
93
596
93
194
167
207
123
695
27
502
339
303
10
165
25
912
103
499
156
209
311
132
66
641
110
405
330
116
21
162
81
124
355
110
280
349
248
73
438
481
32
21
67
317
343
71
207
248
242
180
142
270
255
71
109
529
156
442
198
263
35
98
301
315
234
364
212
340
136
205
105
174
167
342
396
346
153
143
315
96
151
51
54
513
156
86
182
121
1187
243
99
314
55
398
89
135
23
111
172
110
466
186
297
279
391
601
49
168
63
436
135
23
295
233
70
108
125
161
279
17
224
555
336
182
73
465
514
188
22
243
397
23
209
244
87
185
370
325
259
277
176
479
400
291
102
422
673
453
8
88
36
244
60
110
42
289
270
89
237
83
311
615
351
217
408
459
226
179
125
156
71
414
119
163
140
316
76
81
345
537
184
305
379
63
45
75
306
372
102
234
260
235
118
473
370
117
298
232
82
368
122
171
104
27
180
36
179
275
95
273
330
67
536
281
408
362

336
253
289
508
172
395
492
73
259
459
390
286
299
101
317
348
402
107
366
336
230
54
432
145
335
622
261
102
78
428
211
301
123
266
76
225
98
34
10
587
281
441
123
253
414
286
389
210
183
11
195
299
426
150
49
90
97
446
140
325
33
182
154
803
163
117
270
12
170
167
70
746
6
129
154
284
484
174
116
74
452
170
119
436
340
12
120
317
151
132
324
318
195
99
320
53
80
266
408
436
59
173
130
156
53
181
94
862
295
192
385
415
363
409
63
210
219
103
52
440
463
531
453
89
261
138
146
493
234
390
166
89
402
405
162
102
176
184
249
361
290
189
46
351
340
517
345
366
178
201
163
224
213
326
74
132
145
92
312
144
607
321
334
199
540
398
69
64
325
291
40
91
426
422
357
246
164
83
0
540
309
436
292
560
62
247
128
84
246
358
155
327
380
120
16
74
547
380
10
19
115
418
89
142
60
211
137
67
21
84
301
41
371
176
181
602
141
20
418
454
30
7
304
298
312
151
448
334
464
306
294
339
106
260
229
130
59
257
44
234
154
494
154
270
441
410
305
80
196
119
243
185
230
433
424
99
564
353
372
533
328
694
115
304
17

52
358
207
615
93
306
240
257
158
199
303
429
1361
156
431
77
370
488
175
132
467
423
271
104
140
115
403
902
521
123
234
630
114
515
369
266
204
22
257
393
60
399
478
105
255
97
115
172
137
537
31
513
565
325
399
310
65
59
260
77
197
553
200
154
113
455
138
113
222
340
385
352
97
90
198
272
135
213
291
128
129
85
104
152
427
115
189
449
101
244
250
136
370
113
466
101
48
602
205
238
97
144
138
437
809
425
838
501
67
29
273
38
174
291
261
1187
180
18
63
315
143
207
20
189
267
9
314
130
25
11
147
235
227
107
169
62
259
200
250
23
126
523
493
298
111
243
146
190
166
234
185
154
138
286
299
111
437


In [49]:
logisticReg

LogisticRegression()

In [56]:
print(vectorizer)

TfidfVectorizer()


In [74]:
# Preprocess the new data
new_data = ["This is a new news article about politics"]
new_data = [stemming(text) for text in new_data]
new_data = vectorizer.transform(new_data)

# Make predictions on the new data
predictions = logisticReg.predict(new_data)

# Print the predictions
print("Predictions:", predictions)
print(predictions[0]==1?1:0)

Predictions: [1]
True


In [69]:
import pickle

# Pickle the vectorizer object
vectorizer_pickled = pickle.dumps(vectorizer)

# Save the pickled vectorizer to a file
with open("vectorizer.pkl", "wb") as f:
    f.write(vectorizer_pickled)
